Készítsen lekérdezést, amely megjeleníti, hogy szobánként (SZOBA\_FK) hány olyan foglalás történt, ahol a felnőttek száma nagyobb volt a gyermekek számánál!

a. A lekérdezés jelenítse meg a végösszeget is megfelelően jelölve!

In [4]:
SELECT (   
    CASE GROUPING(Szoba_fk)
    WHEN 0 THEN CAST(Szoba_fk AS nvarchar)
    WHEN 1 THEN 'Összesen' END
) AS 'Szoba_FK', 
        Count(*) AS 'Darabszám'
FROM foglalas
WHERE Felnott_szam > gyermek_szam
GROUP BY Rollup(Szoba_fk)

(190 rows affected)

Total execution time: 00:00:00.013

Szoba_FK,Darabszám
1,7
2,2
3,3
4,5
5,2
6,5
7,6
8,5
9,4
10,4
